# KillSwitch Project: Design Candidate Guide RNAs

## Choose an essential gene

Used Yeastmine to search for genes whose "Gene Phenotype Summary" said they were an "Essential gene". Query code in python below:

In [ ]:
from intermine.webservice import Service
service = Service("http://yeastmine.yeastgenome.org/yeastmine/service")
query = service.new_query("Gene")
query.add_view(
    "primaryIdentifier", "secondaryIdentifier", "organism.shortName", "symbol",
    "name", "phenotypeSummary"
)
query.add_constraint("phenotypes.strainBackground", "=", "W303", code = "A")
query.add_constraint("phenotypeSummary", "LIKE", "Essential gene*", code = "B")

columns = ["primaryIdentifier", "secondaryIdentifier", "organism.shortName", "symbol", "name", "phenotypeSummary"]

output = '<html><table>'
output += '<tr>' + ' '.join(['<th>'+k+'</th>' for k in columns]) + '</tr>'
for row in query.rows():
    output += '<tr>' + ' '.join(['<td>'+str(row[c])+'</td>' for c in columns]) + '</tr>'
output += '</table></html>'
HTML(output)

Out of those results, I selected the following 2 genes:

- **[LCB1 / YMR296C](http://www.yeastgenome.org/locus/S000004911/overview):** This is responsible for sphingolopid synthesis, defects in this gene result in inviablity. I picked it because it said it causes "shmoo formation" which sounded cool.
- **[RPT5 / YOR117W](http://www.yeastgenome.org/locus/S000005643/overview):** This is one of the 6 ATPases, mutations cause short lifespan, shorter telomeres.

## Find candidate PAM sites & design guide RNA

In [63]:
import coral
from collections import defaultdict

In [65]:
sites = defaultdict(defaultdict)

In [73]:
lcb1_promoter = coral.database.get_yeast_promoter_ypa('LCB1')
lcb1_promoter.name = 'LCB1-Promoter'
lcb1_promoter.features.append(coral.Feature('LCB1-Promoter', 0, len(lcb1_promoter), 'promoter'))
lcb1_promoter.display()

In [71]:
lcb1_cds = coral.database.fetch_yeast_locus_sequence('LCB1')
lcb1_cds.name = 'LCB1'
lcb1_cds.features.append(coral.Feature('LCB1', 0, len(lcb1_cds), 'CDS'))
lcb1_cds.display()

In [78]:
# find 'NGG' sites, list the last 10 in the promoter (forward direction)
lcb1_promoter.locate('GG')[0][-10:]

[718, 738, 809, 810, 811, 824, 883, 953, 983, 984]

In [124]:
?coral.Feature

In [125]:
site1 = 983
lcb1_site1 = site1
lcb1_promoter.features.append(coral.Feature('LCB1-pam1', site1-20, 23, 'site'))
lcb1_g1 = lcb1_promoter[site1-20:site1+3]
lcb1_g1.name = 'LCB1-guide1'
lcb1_g1.features.append(coral.Feature('LCB1-guide1', 0, 23, 'misc_RNA'))
lcb1_g1

ATACACATTTTGCTCGTGTAGGG
TATGTGTAAAACGAGCACATCCC

In [126]:
site2 = 953-1
lcb1_site2 = site2
lcb1_promoter.features.append(coral.Feature('LCB1-pam2', site2-20, 23, 'site'))
lcb1_g2 = lcb1_promoter[site2-20:site2+3]
lcb1_g2.name = 'LCB1-guide1'
lcb1_g2.features.append(coral.Feature('LCB1-guide2', 0, 23, 'misc_RNA'))
lcb1_g2

GAGCGCTCTAACCTTCTGCCTGG
CTCGCGAGATTGGAAGACGGACC

In [127]:
template = lcb1_promoter + lcb1_cds
template[site2-50:].display()

## Make mockup plasmids for each guide RNA

In [99]:
# import template plasmid
orig_plasmid = coral.seqio.read_dna('./pmod8-pgalz4-urgr.gb')
orig_plasmid.display()

In [108]:
target = orig_plasmid.select_features('Target Sequence')[0]
print 'len:', target.stop - target.start
orig_plasmid[target.start:target.stop]

len: 20


NNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNN

In [109]:
def feat(dna, name):
    return dna.select_features(name)[0]

In [144]:
def zoom(plasmid):
    zoom = (feat(plasmid,'ASBV1').start, feat(plasmid,'gRNA Handle').stop)
    pz = plasmid[zoom[0]:zoom[1]]
    pz.name += ' (zoom ASBV1..gRNA)'
    return pz.display()

In [145]:
zoom(orig_plasmid)

In [116]:
backbone = orig_plasmid.rotate(target.start)[target.stop-target.start:]
backbone.display()

In [150]:
lcb1_g1_mockup = (lcb1_g1+backbone).circularize()
lcb1_g1_mockup.name = 'pMOD6-killswitch-ASBV1-g1'
zoom(lcb1_g1_mockup.rotate(200)) # need to rotate so the zoomed region doesn't wrap around 0

In [149]:
lcb1_g2_mockup = (lcb1_g2+backbone).circularize()
lcb1_g2_mockup.name = 'pMOD6-killswitch-ASBV1-g2'
zoom(lcb1_g2_mockup.rotate(200)) # need to rotate so the zoomed region doesn't wrap around 0

## Design primers

In [242]:
# LCB1 gene, guide for PAM site #1:
lcb1_g1

ATACACATTTTGCTCGTGTAGGG
TATGTGTAAAACGAGCACATCCC

In [243]:
lcb1_g1_fwd = coral.design.primer(backbone.flip(), overhang=lcb1_g1.flip())
lcb1_g1_fwd

Primer: ccctacacgagcaaaatgtgtatGGGAGATGGGACGGGC Tm: 67.37

In [244]:
lcb1_g1_rev = coral.design.primer(backbone, overhang=lcb1_g1)
lcb1_g1_rev

Primer: atacacattttgctcgtgtagggGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGG Tm: 64.35

In [245]:
coral.reaction.pcr(lcb1_g1_fwd.primer().to_ds(), lcb1_g1_fwd, lcb1_g1_rev)

PrimingError: Primer 2 does not bind the template

This works, if I create the primers manually (copy/pasted the overhang sequences from Benchling):

In [246]:
fwd = coral.Primer(anneal=lcb1_g1, overhang=coral.DNA('GCCCGTCCCATCTCCC'), tm=65)
fwd

Primer: gcccgtcccatctcccATACACATTTTGCTCGTGTAGGG Tm: 65.00

In [247]:
rev = coral.Primer(anneal=lcb1_g1.reverse_complement(), overhang=coral.DNA('gttttagagctagaaatagcaagttaaaataaggc').reverse_complement(), tm=65)
rev

Primer: gccttattttaacttgctatttctagctctaaaacCCCTACACGAGCAAAATGTGTAT Tm: 65.00

In [249]:
coral.reaction.pcr(fwd.primer().to_ds(), fwd, rev)

GCCCGTCCCATCTCCCATACACATTTTGCTCGTGTAGGGGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGC
CGGGCAGGGTAGAGGGTATGTGTAAAACGAGCACATCCCCAAAATCTCGATCTTTATCGTTCAATTTTATTCCG